Importing libraries

In [26]:
import csv
import pandas as pd
import numpy as np
import sys
from scipy import stats
import matplotlib.pyplot as plt

First, selected only providers in the US and then dropped columns that we dont want to use.

In [27]:
# data1 = pd.read_csv('Dataset.csv', encoding='utf-8')
data = pd.read_csv('Dataset.csv', nrows = 200, encoding='utf-8')
 
data = data[data.nppes_provider_country=='US']


In [28]:
#npi=National Provider Identifier
#nppes_provider_last_org_name=Last Name/Organization Name of the Provider
#nppes_provider_mi= Middle Initial of the Provider
#description_flag = Source of Provider Specialty
data.drop(['npi',
           'nppes_provider_last_org_name',
           'nppes_provider_first_name',
           'nppes_provider_mi',
           'nppes_provider_street1',
           'nppes_provider_street2',
           'nppes_provider_zip4',
           'nppes_provider_country',
           'description_flag',
           'ge65_suppress_flag',
           'total_claim_count_ge65',
           'total_30_day_fill_count_ge65',
           'total_drug_cost_ge65',
           'total_day_supply_ge65',
           'bene_count_ge65_suppress_flag',
           'bene_count_ge65',
           'mapd_suppress_flag',
           'pdp_suppress_flag',
           'lis_suppress_flag',
           'nonlis_suppress_flag',
           'opioid_prescriber_rate',
           'hrm_ge65_suppress_flag',
           'hrm_claim_count_ge65',
           'hrm_drug_cost_ge65',
           'hrm_bene_ge65_suppress_flag',
           'hrm_bene_count_ge65',
           'antipsych_ge65_suppress_flag',
           'antipsych_claim_count_ge65',
           'antipsych_drug_cost_ge65',
           'antipsych_bene_ge65_suppress_flg',
           'antipsych_bene_count_ge65',
           'mapd_claim_count',                   
           'mapd_drug_cost',                      
           'pdp_claim_count',                     
            'pdp_drug_cost',                      
            'lis_claim_count',                      
            'lis_drug_cost',                        
            'nonlis_claim_count',                   
            'nonlis_drug_cost',                     
            'opioid_claim_count',                  
            'opioid_drug_cost',                     
            'opioid_day_supply',                    
            'opioid_bene_count',                   
            'antibiotic_claim_count',              
            'antibiotic_drug_cost',                 
            'antibiotic_bene_count',
             'beneficiary_age_less_65_count',
             'beneficiary_age_65_74_count',
             'beneficiary_age_75_84_count',
             'beneficiary_age_greater_84_count', 'nppes_provider_zip5', 'nppes_provider_city'
            ],axis = 1,inplace = True)

data.index = range(len(data))
data.replace({',':'','\$':''},regex= True,inplace = True)

Declaring descriptive attributes as strings and total costs to ints

In [29]:
data['nppes_credentials'] = data['nppes_credentials'].astype(str)
data['nppes_provider_gender'] = data['nppes_provider_gender'].astype(str)
data['nppes_entity_code'] = data['nppes_entity_code'].astype(str)
data['nppes_provider_state'] = data['nppes_provider_state'].astype(str)
data['specialty_description'] = data['specialty_description'].astype(str)
data['total_drug_cost'] = data['total_drug_cost'].astype(float)
data['brand_drug_cost'] = data['brand_drug_cost'].astype(float)
data['generic_drug_cost'] = data['generic_drug_cost'].astype(float)

data['other_drug_cost'] = data['other_drug_cost'].astype(float)
data['nppes_provider_gender'] = data['nppes_provider_gender'].replace('nan','Org')
data['nppes_credentials'] = data['nppes_credentials'].replace('nan', 'Org')

<font color="red">When the provider is registered in NPPES as an individual
(entity_type_code = “I”), this is the provider’s last name. When the provider is registered as an
organization (nppes_entity_code = “O”), this is the organization name.</font>

<p><b>medicare_prvdr_enroll_status</b> – 
<font color="blue">A status to indicate whether the prescriber is enrolled in the
Medicare Program.
“E” = Providers who are enrolled in the Medicare Program as of end of the data reporting year.
“N” = Providers who are not enrolled in the Medicare Program as of the end of the data
reporting year.
“O” = Providers who did not wish to enroll in the Medicare program as of end of the data
reporting year but have signed a written affidavit to opt out that states neither the
provider nor the beneficiary can receive payment from Medicare for services performed.</font><p>

Replacing string features with categorical bins

In [30]:
data.loc[data.nppes_provider_gender == 'M', 'nppes_provider_gender'] = 0
data.loc[data.nppes_provider_gender == 'F', 'nppes_provider_gender'] = 1
data.loc[data.nppes_provider_gender == 'Org', 'nppes_provider_gender'] = 2
#nppes_entity_code =Entity Type of the Provider
data.loc[data.nppes_entity_code == 'O', 'nppes_entity_code'] = 0
data.loc[data.nppes_entity_code == 'I', 'nppes_entity_code'] = 1
data.loc[data.medicare_prvdr_enroll_status == 'O', 'medicare_prvdr_enroll_status'] = 0
data.loc[data.medicare_prvdr_enroll_status == 'E', 'medicare_prvdr_enroll_status'] = 1
data.loc[data.medicare_prvdr_enroll_status == 'N', 'medicare_prvdr_enroll_status'] = 2
state_list = data.nppes_provider_state.unique()
specialty_list = data.specialty_description.unique()
credentials_list = data.nppes_credentials.unique()
unique_vars = {v: k for k, v in enumerate(np.unique(state_list))}
data['nppes_provider_state'] = np.array([unique_vars[i] for i in data['nppes_provider_state']])
unique_vars = {v: k for k, v in enumerate(np.unique(specialty_list))}
data['specialty_description'] = np.array([unique_vars[i] for i in data['specialty_description']])
unique_vars = {v: k for k, v in enumerate(np.unique(credentials_list))}
data['nppes_credentials'] = np.array([unique_vars[i] for i in data['nppes_credentials']])

      

Removing the columns with NaN values because we need them to calculate the mean.


In [31]:
def remove_NaN_Values(column,data):
    data_temp=data.dropna(subset=[column])
    return data_temp[column].mean()
    

total_claim_count – The number of Medicare Part D claims. This includes original prescriptions and
refills. Aggregated records based on total_claim_count fewer than 11 are not included in the data file.


bene_count – The total number of unique Medicare Part D beneficiaries with at least one claim for the
drug. Counts fewer than 11 are suppressed and are indicated by a blank.

Replacing missing values in non-string columns and calculating ratio columns. 

In [36]:
average_total =remove_NaN_Values('total_claim_count',data)
average_bene = remove_NaN_Values('bene_count',data)
print(average_total,average_bene)
bene_ratio = average_bene / average_total
round(bene_ratio * data['total_claim_count'],0)
print(bene_ratio)


average_age = remove_NaN_Values('average_age_of_beneficiaries',data)
average_fem = remove_NaN_Values('beneficiary_female_count',data)
average_fem_ratio = average_fem / average_bene
average_male_ratio = 1- average_fem_ratio

avg_bene1 = remove_NaN_Values('bene_count',data)

avg_white = remove_NaN_Values('beneficiary_race_white_count',data)
avg_white_ratio = avg_white / avg_bene1

avg_black = remove_NaN_Values('beneficiary_race_black_count',data)
avg_black_ratio = avg_black / avg_bene1

avg_asian = remove_NaN_Values('beneficiary_race_asian_pi_count',data)
avg_asian_ratio = avg_asian / avg_bene1

avg_hisp = remove_NaN_Values('beneficiary_race_hispanic_count',data)
avg_hisp_ratio = avg_hisp / avg_bene1

avg_nat = remove_NaN_Values('beneficiary_race_nat_ind_count',data)
avg_nat_ratio = avg_nat / avg_bene1

avg_other = remove_NaN_Values('beneficiary_race_other_count',data)
avg_other_ratio = avg_other / avg_bene1

avg_nondual = remove_NaN_Values('beneficiary_nondual_count',data)
avg_nondual_ratio = avg_nondual / average_bene
avg_dual = remove_NaN_Values('beneficiary_dual_count',data)
avg_dual_ratio = avg_dual / average_bene


avg_risk = remove_NaN_Values('beneficiary_average_risk_score',data)

1050.18 121.33540372670808
0.11553772089233091


Checking datatypes of the columns

In [37]:
data.dtypes

nppes_credentials                    int32
nppes_provider_gender               object
nppes_entity_code                   object
nppes_provider_state                 int32
specialty_description                int32
medicare_prvdr_enroll_status        object
total_claim_count                  float64
total_30_day_fill_count             object
total_drug_cost                    float64
total_day_supply                    object
bene_count                         float64
brand_suppress_flag                 object
brand_claim_count                   object
brand_drug_cost                    float64
generic_suppress_flag               object
generic_claim_count                 object
generic_drug_cost                  float64
other_suppress_flag                 object
other_claim_count                  float64
other_drug_cost                    float64
average_age_of_beneficiaries       float64
beneficiary_female_count           float64
beneficiary_male_count             float64
beneficiary

Certain columns have object as their datatype. We need to change it to float/int to perform mathematical operations. Changing to float.

In [38]:
data['total_claim_count']=data['total_claim_count'].astype('float64')
data['bene_count']=data['bene_count'].astype('float64')
data['beneficiary_nondual_count']=data['beneficiary_nondual_count'].astype('float64')
         


In [39]:
print(data['total_claim_count'][1])
print(bene_ratio)

84.0
0.11553772089233091


In [40]:
print(round(bene_ratio * data['total_claim_count'],0))

0        2.0
1       10.0
2        3.0
3        3.0
4        3.0
5        2.0
6        2.0
7        3.0
8        6.0
9        6.0
10      13.0
11       1.0
12       6.0
13       4.0
14       3.0
15       1.0
16       2.0
17      26.0
18       7.0
19       4.0
20       1.0
21      15.0
22       2.0
23     520.0
24       9.0
25       2.0
26       2.0
27      20.0
28      10.0
29      21.0
       ...  
170      2.0
171      2.0
172     11.0
173     21.0
174      2.0
175    210.0
176      2.0
177    138.0
178     13.0
179      1.0
180      6.0
181     10.0
182     89.0
183     34.0
184     53.0
185      5.0
186      4.0
187      4.0
188      7.0
189     38.0
190     24.0
191     18.0
192     52.0
193      8.0
194    310.0
195      4.0
196     58.0
197    211.0
198    174.0
199    188.0
Name: total_claim_count, Length: 200, dtype: float64


Using the mean to fill in missing values. Then calculated relevant ratios so that our features will make sense for modeling. 
Dropped features that are no longer necessary after generating ratios.

In [41]:
data.loc[data.bene_count.isnull(), 'bene_count'] = round(bene_ratio * data['total_claim_count'],0)
data.loc[data.beneficiary_female_count.isnull(), 'beneficiary_female_count'] = round(average_fem_ratio * data['bene_count'],0)
data.loc[data.beneficiary_male_count.isnull(), 'beneficiary_male_count'] = round(average_male_ratio * data['bene_count'],0)
data.loc[data.average_age_of_beneficiaries.isnull(), 'average_age_of_beneficiaries'] = round(average_age,0)
data.loc[data.beneficiary_race_white_count.isnull(), 'beneficiary_race_white_count'] = round(avg_white_ratio * data['bene_count'],0)
data.loc[data.beneficiary_race_black_count.isnull(), 'beneficiary_race_black_count'] = round(avg_black_ratio * data['bene_count'],0)
data.loc[data.beneficiary_race_asian_pi_count.isnull(), 'beneficiary_race_asian_pi_count'] = round(avg_asian_ratio * data['bene_count'],0)
data.loc[data.beneficiary_race_hispanic_count.isnull(), 'beneficiary_race_hispanic_count'] = round(avg_hisp_ratio * data['bene_count'],0)
data.loc[data.beneficiary_race_nat_ind_count.isnull(), 'beneficiary_race_nat_ind_count'] = round(avg_nat_ratio * data['bene_count'],0)
data.loc[data.beneficiary_race_other_count.isnull(), 'beneficiary_race_other_count'] = round(avg_other_ratio * data['bene_count'],0)
data.loc[data.beneficiary_nondual_count.isnull(), 'beneficiary_nondual_count'] = round(avg_nondual_ratio * data['bene_count'],0)
data.loc[data.beneficiary_dual_count.isnull(), 'beneficiary_dual_count'] = round(avg_dual_ratio * data['bene_count'],0)
data.loc[data.beneficiary_average_risk_score.isnull(), 'beneficiary_average_risk_score'] = avg_risk

Assign new columns to a DataFrame, returning a new object (a copy) with all the original columns in addition to the new ones.

In [42]:


data = data.assign(beneficiary_female_count = lambda x: (x.beneficiary_female_count /x.bene_count))
data['gender_ratio'] = data['beneficiary_female_count']
data = data.drop(['beneficiary_female_count','beneficiary_male_count'], axis = 1)
data = data.assign(beneficiary_race_white_count = lambda x: (x.beneficiary_race_white_count /x.bene_count))
data['white_ratio'] = data['beneficiary_race_white_count']
data = data.assign(beneficiary_race_black_count = lambda x: (x.beneficiary_race_black_count /x.bene_count))
data['black_ratio'] = data['beneficiary_race_black_count']
data = data.assign(beneficiary_race_asian_pi_count = lambda x: (x.beneficiary_race_asian_pi_count /x.bene_count))
data['asian_ratio'] = data['beneficiary_race_asian_pi_count']
data = data.assign(beneficiary_race_hispanic_count = lambda x: (x.beneficiary_race_hispanic_count /x.bene_count))
data['hispanic_ratio'] = data['beneficiary_race_hispanic_count']
data = data.assign(beneficiary_race_nat_ind_count = lambda x: (x.beneficiary_race_nat_ind_count /x.bene_count))
data['nat_ind_ratio'] = data['beneficiary_race_nat_ind_count']
data = data.assign(beneficiary_race_other_count = lambda x: (x.beneficiary_race_other_count /x.bene_count))
data['other_ratio'] = data['beneficiary_race_other_count']
data = data.drop(['beneficiary_race_white_count','beneficiary_race_black_count', 'beneficiary_race_asian_pi_count', 'beneficiary_race_hispanic_count', 'beneficiary_race_nat_ind_count', 'beneficiary_race_other_count'], axis = 1)
data = data.assign(beneficiary_nondual_count = lambda x: (x.beneficiary_nondual_count /x.bene_count))


In [43]:
data['nondual_insure_ratio'] = data['beneficiary_nondual_count']
data = data.drop(['beneficiary_dual_count', 'beneficiary_nondual_count'], axis =1)

Dropping rows where total claim count equals other claim count

In [44]:
for index, row in data.iterrows():
    if row['total_claim_count'] == row['other_claim_count']:
        data = data.drop(index)

Dropping any rows where brand or generic counts are missing

In [45]:
data_est = data
data = data.drop(data[data.brand_suppress_flag == '#'].index)
data = data.drop(data[data.brand_suppress_flag == '*'].index)
data = data.drop(data[data.generic_suppress_flag == '#'].index)
data = data.drop(data[data.generic_suppress_flag == '*'].index)

<html><p><b>data_est['brand_suppress_flag']</b></p><font color="green">A flag indicating the reason the brand_claim_count and brand_drug_cost
variables are suppressed.
<p>“*” = Primary suppressed due to brand_claim_count between 1 and 10.</p>
<p>“#” = Counter suppressed because the claim count from at least one of the corresponding claim
count categories (generic_claim_count or other_claim_count) is between 1 and 10.</p> 
Counter suppression prevents the disclosure of a primary suppressed value when all categories sum
to the total value.<html></font>

In [46]:
data_est['brand_suppress_flag']

0        *
1      NaN
2        *
3        *
4        *
5      NaN
6        *
7        #
8      NaN
9      NaN
10       #
11     NaN
12       *
13     NaN
14       *
15       *
16       *
17       #
18       #
19       *
20       *
21     NaN
22       #
23     NaN
24     NaN
25       *
26       *
27     NaN
28     NaN
29     NaN
      ... 
170      *
171      *
172    NaN
173    NaN
174      *
175    NaN
176      #
177      #
178    NaN
179      *
180    NaN
181    NaN
182      #
183    NaN
184    NaN
185      *
186      *
187      *
188      *
189      #
190      #
191    NaN
192    NaN
193    NaN
194    NaN
195      *
196      #
197      #
198    NaN
199      #
Name: brand_suppress_flag, Length: 200, dtype: object

<b>We have to change the datatypes of these two columns I checked because we are using them for subtraction operation so they have to be converted from object datatype to float64. </b>

In [47]:
print("datatype of column total_claim_count",data_est['total_claim_count'].dtypes)
print("datatype of column brand_claim_count",data_est['brand_claim_count'].dtypes)
print("datatype of column generic_claim_count",data_est['generic_claim_count'].dtypes)
data_est['brand_claim_count']=data_est['brand_claim_count'].astype('float64')
data_est['generic_claim_count']=data_est['generic_claim_count'].astype('float64')


datatype of column total_claim_count float64
datatype of column brand_claim_count object
datatype of column generic_claim_count object


In [48]:
data_est = data_est.drop(data_est[(data_est.brand_suppress_flag == '*') & (data_est.generic_suppress_flag == '*')].index)
data_est.loc[data_est.brand_suppress_flag=='*', 'brand_claim_count'] = 1
data_est.loc[data_est.brand_suppress_flag=='*', 'brand_drug_cost'] = 1
data_est.loc[data_est.generic_suppress_flag =='*', 'generic_claim_count'] = 1
data_est.loc[data_est.generic_suppress_flag =='*', 'generic_drug_cost'] = 1
data_est.loc[data_est.other_suppress_flag =='*', 'other_claim_count'] = 1

data_est.loc[data_est.other_suppress_flag =='#', 'other_claim_count'] = str(data_est.total_claim_count - data_est.brand_claim_count - data_est.generic_claim_count)
data_est.loc[data_est.other_suppress_flag =='*', 'other_drug_cost'] = 1
data_est.loc[data_est.other_suppress_flag=='#', 'other_drug_cost'] = data_est.total_drug_cost - data_est.brand_drug_cost - data_est.generic_drug_cost
data_est.loc[data_est.brand_suppress_flag =='#', 'brand_claim_count'] = data_est.total_claim_count - data_est.generic_claim_count - data_est.other_claim_count
data_est.loc[data_est.brand_suppress_flag =='#', 'brand_drug_cost'] = data_est.total_drug_cost - data_est.generic_drug_cost - data_est.other_drug_cost
data_est.loc[data_est.generic_suppress_flag =='#', 'generic_claim_count'] = data_est.total_claim_count - data_est.brand_claim_count - data_est.other_claim_count
data_est.loc[data_est.generic_suppress_flag =='#', 'generic_drug_cost'] = data_est.total_drug_cost - data_est.brand_drug_cost - data_est.other_drug_cost

data = data.assign(brand_drug_cost = lambda x: (x.brand_drug_cost /(x.total_drug_cost - x.other_drug_cost)))
data['brand_drug_cost_ratio'] = data['brand_drug_cost']
data = data.drop(data[data.total_drug_cost == 0].index)
data = data.drop(['brand_drug_cost','generic_drug_cost', 'other_drug_cost'], axis = 1)
data_est = data_est.assign(brand_drug_cost = lambda x: (x.brand_drug_cost /(x.total_drug_cost - x.other_drug_cost)))
data_est['brand_drug_cost_ratio'] = data_est['brand_drug_cost']
data_est = data_est.drop(data_est[data_est.total_drug_cost == 0].index)
data_est = data_est.drop(['brand_drug_cost','generic_drug_cost', 'other_drug_cost'], axis = 1)

In [49]:
data_est['brand_claim_count']/(data_est['total_claim_count']-data_est['other_claim_count'])

0      0.055556
1      0.154762
2      0.034483
3      0.038462
4      0.037037
5      0.000000
6      0.076923
7      0.965517
8      0.363636
9      0.416667
10     0.330435
11     0.000000
12     0.017857
13     0.542857
14     0.043478
16     0.052632
17     0.265766
18     0.410714
19     0.026316
21     0.335821
22     0.950000
23     0.162522
24     0.597561
25     0.052632
26     0.050000
27     0.194118
28     0.393258
29     0.413043
30     0.027027
31     0.120253
         ...   
167    0.595699
168    0.227589
169    0.016129
172    0.489362
173    0.627027
174    0.076923
175    0.265928
176    0.952381
177    0.437448
178    0.159292
180    0.207547
181    0.156627
182    0.159061
183    0.265101
184    0.409091
185    0.023810
186    0.030303
187    0.027778
188    0.015625
189    0.385802
190    0.103448
191    0.272727
192    0.039906
193    0.152778
194    0.772473
195    0.030303
196    0.149402
197    0.131435
198    0.094040
199    0.183159
Length: 193, dtype: floa

In [50]:

data['brand_claim_count']=data['brand_claim_count'].astype('float64')
data['total_claim_count']=data['total_claim_count'].astype('float64')
data['other_claim_count']=data['other_claim_count'].astype('float64')


Calculating brand percentage of total and adding it to main dataframe

In [51]:
data_est = data_est.assign(brand_ratio = lambda x: (x.brand_claim_count /(x.total_claim_count - x.other_claim_count)))
data = data.assign(brand_ratio = lambda x: (x.brand_claim_count /(x.total_claim_count - x.other_claim_count)))
data_est = data_est.drop(['brand_claim_count','generic_claim_count'], axis = 1)
data = data.drop(['brand_claim_count','generic_claim_count'], axis = 1)

Assigning quartile to each brand ratio as target variable, and dropping brand_ratio for use in classification models

In [52]:
data['quartile'] = pd.qcut(data['brand_ratio'], 4, ['1','2','3','4'])
data_est['quartile'] = pd.qcut(data_est['brand_ratio'], 4, ['1','2','3','4'])
data = data.drop(['brand_ratio','brand_suppress_flag', 'generic_suppress_flag', 'other_suppress_flag'], axis = 1)
data_est = data_est.drop(['brand_ratio', 'brand_suppress_flag', 'generic_suppress_flag', 'other_suppress_flag'], axis = 1)
data.index = range(1,len(data) + 1)
data_est.index = range(1,len(data_est) + 1)

Standardizing values of non-string columns

In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[['total_claim_count', 'total_30_day_fill_count',
       'total_day_supply','total_drug_cost', 'bene_count',
       'brand_drug_cost_ratio',
       'other_claim_count', 'average_age_of_beneficiaries', 'beneficiary_average_risk_score',
       'gender_ratio', 'white_ratio', 'black_ratio', 'asian_ratio',
       'hispanic_ratio', 'nat_ind_ratio', 'other_ratio',
       'nondual_insure_ratio']] = scaler.fit_transform(data[['total_claim_count', 'total_30_day_fill_count',
       'total_day_supply','total_drug_cost', 'bene_count',
       'brand_drug_cost_ratio',
       'other_claim_count', 'average_age_of_beneficiaries',
       'beneficiary_average_risk_score',
       'gender_ratio', 'white_ratio', 'black_ratio', 'asian_ratio',
       'hispanic_ratio', 'nat_ind_ratio', 'other_ratio',
       'nondual_insure_ratio']])

data_est[['total_claim_count', 'total_30_day_fill_count',
       'total_day_supply','total_drug_cost', 'bene_count',
       'brand_drug_cost_ratio',
       'other_claim_count', 'average_age_of_beneficiaries',
       'beneficiary_average_risk_score',
       'gender_ratio', 'white_ratio', 'black_ratio', 'asian_ratio',
       'hispanic_ratio', 'nat_ind_ratio', 'other_ratio',
       'nondual_insure_ratio']] = scaler.fit_transform(data_est[['total_claim_count', 'total_30_day_fill_count',
       'total_day_supply','total_drug_cost', 'bene_count',
       'brand_drug_cost_ratio',
       'other_claim_count', 'average_age_of_beneficiaries',
       'beneficiary_average_risk_score',
       'gender_ratio', 'white_ratio', 'black_ratio', 'asian_ratio',
       'hispanic_ratio', 'nat_ind_ratio', 'other_ratio',
       'nondual_insure_ratio']])

Splitting both datasets into train, validate, and test sets. Assigning X and Y vectors

In [54]:
train, validate, test = np.split(data.sample(frac=1), [int(.7*len(data)), int(.9*len(data))])
X_data_train = train.iloc[:,0:22].values
Y_data_train = train.iloc[:,23].values

train, validate, test = np.split(data_est.sample(frac=1), [int(.7*len(data_est)), int(.9*len(data_est))])
X_est_train = train.iloc[:,0:22].values
Y_est_train = train.iloc[:,23].values